# Weather AUS

Importa as bibliotecas necessárias:

 - pandas e numpy: Para manipulação de dados e operações numéricas.

 - tensorflow e keras: Para construir, compilar e treinar o modelo de RNN.

 - sklearn.preprocessing e sklearn.model_selection: Para normalização e divisão dos dados.

 - matplotlib.pyplot: Para visualização dos resultados.

In [ ]:
import os
import random
import pandas as pd
import numpy as np
import random
import tensorflow as tf
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from tensorflow import keras
from keras.optimizers import Adam, RMSprop
from keras.models import Sequential
from keras.layers import SimpleRNN, LSTM, Dense
import matplotlib.pyplot as plt


# Definir a seed para reprodutibilidade
seed = 0
np.random.seed(seed)
tf.random.set_seed(seed)
random.seed(seed)
os.environ['PYTHONHASHSEED'] = str(seed)

initializer = tf.keras.initializers.GlorotUniform(seed=seed)

tf.config.experimental.enable_op_determinism()

Carrega o conjunto de dados weatherAUS.csv.

Filtra os dados para incluir apenas informações da localidade Albury.

Seleciona as colunas Humidity9am e Humidity3pm como variáveis de entrada e saída.

In [ ]:
# Carregar dados do arquivo CSV
weather_data = pd.read_csv('weatherAUS.csv')

# Extrair colunas de interesse e linhas da localidade Albury
data = weather_data.loc[weather_data['Location'] == 'Albury']
data = data.iloc[:,13:15]
data

,Humidity9am,Humidity3pm
0,71.0,22.0
1,44.0,25.0
2,38.0,30.0
3,45.0,16.0
4,82.0,33.0
...,...,...
142188,59.0,27.0
142189,51.0,24.0
142190,56.0,21.0
142191,53.0,24.0


# Pré-Processamento

 - Remove valores ausentes (NaN) do conjunto de dados.

 - Converte as variáveis de entrada (Humidity9am) e saída (Humidity3pm) em arrays NumPy.

 - Normaliza os dados para o intervalo [0, 1] usando MinMaxScaler.

In [ ]:
# Pre-processamento dos dados
data_cleaned = data.dropna()
x = data_cleaned['Humidity9am'].values.reshape(-1, 1)
y = data_cleaned['Humidity3pm'].values.reshape(-1, 1)

# Normalização dos dados
scaler_x = MinMaxScaler()
scaler_y = MinMaxScaler()
x_scaled = scaler_x.fit_transform(x)
y_scaled = scaler_y.fit_transform(y)

# Divisão em conjunto de treinamento e teste
x_train, x_test, y_train, y_test = train_test_split(x_scaled, y_scaled, test_size=0.2, random_state=42)

# Ajuste do shape para RNN (supondo uma janela de tempo de 4)
time_steps = 4
samples_train = x_train.shape[0] // time_steps  # Número de amostras ajustadas para múltiplos de time_steps

# Redimensionando os dados para RNN
x_train_rnn = x_train[:samples_train * time_steps].reshape(samples_train, time_steps, 1)
y_train_rnn = y_train[:samples_train * time_steps].reshape(samples_train, time_steps, 1)

# Test set - mantendo sequência temporal unitária
x_test_rnn = x_test.reshape(x_test.shape[0], 1, 1)

 - Divide os dados em conjuntos de treinamento e teste (80% e 20%).

 - Redimensiona os dados de entrada para o formato esperado pela RNN ([amostras, timesteps, features]).

Modelo da RNN

Define um modelo sequencial composto por:

 - Uma camada SimpleRNN com 20 neurônios e ativação ReLU.

 - Três camadas densas para refinar as previsões.

 - Uma camada de saída com um único neurônio (regressão).

In [ ]:
# Criar o modelo LSTM
model = Sequential([
    SimpleRNN(units=20, activation = 'relu', return_sequences=True, input_shape=(4, 1)),
    Dense(units=10, activation = 'relu'),
    Dense(units=5, activation = 'relu'),
    Dense(1)
])

/usr/local/lib/python3.11/dist-packages/keras/src/layers/rnn/rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Compila e treina

 - Define o otimizador Adam com uma taxa de aprendizado personalizada.
 - Compila o modelo com a função de perda mean_squared_error.
 - Treina o modelo por 20 épocas com tamanho de batch de 32.

In [ ]:
learning_rate = 0.00001
optimizer = Adam(learning_rate=learning_rate)

# Compilar modelo
model.compile(optimizer=optimizer, loss='mean_squared_error')

# Treinar modelo
history = model.fit(x_train_rnn, y_train_rnn, epochs=20, batch_size=32, verbose=1)

Epoch 1/2
862/862 ━━━━━━━━━━━━━━━━━━━━ 7s 3ms/step - loss: 0.2108
Epoch 2/2
862/862 ━━━━━━━━━━━━━━━━━━━━ 3s 3ms/step - loss: 0.1400


# Previsão

In [ ]:
# Predição
Y_predicted = model.predict(x_test)

# Desescalonar os resultados
Y_test_descaled = scaler_y.inverse_transform(y_test)

# Reshape
Y_predicted = Y_predicted.reshape(Y_predicted.shape[0],Y_predicted.shape[1])
Y_predicted_descaled = scaler_y.inverse_transform(Y_predicted)

862/862 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step


 - Define uma função para calcular o coeficiente de determinação (R²).

 - Avalia a performance do modelo ao comparar os valores previstos com os reais.

In [ ]:
# Calcular R-squared
def r_squared(y_actual, y_predict):
    corr = np.corrcoef(y_actual.flatten(), y_predict.flatten())[0, 1]
    return corr ** 2

r2_score = r_squared(Y_test_descaled, Y_predicted_descaled)
print(f'R-squared: {r2_score}')

R-squared: 0.4408950167807713
